# Serial Dilution
Get familiar with science-jubilee functionality using this serial dilution example notebook! We'll mix colors using a pipette tool.

### Before Starting:
Clear any existing items off the bed of your machine!

### Zero the Extruder Position
Our syringe tool doesn't have any limit switches; for liquid handling, that meens the syringe plunger should start at 0 mL when the machine starts. If your syringe is not already there, you can manually move to 0 and restart your machine. 

To do so, open Duet Web Control and pickup the syringe. Then, send 'extrude' commands in small increments until the plunger is at 0. Park the syringe, and restart the machine.

In [ ]:
# Import the tools we'll need!
from science_jubilee.Machine import Machine
from science_jubilee.tools.Syringe import Syringe
from science_jubilee.tools.AS7341 import AS7341
from science_jubilee.decks import Deck
import json

In [ ]:
# Connect to the machine and load our deck configuration
m = Machine(address='192.168.1.2')
deck = m.load_deck('POSE-calibrated-deck')

In [ ]:
# m.home_all() # if you need to, uncomment this line to home the machine

### Zero the Z Position
By default, we leave quite a bit of distance between the bed and the syringe's '0' position because we can attach tips of different lengths to the syringe. To start, we need to calibrate the syringe tip to the bed.

If you haven't calibrated the Z position yet, **do so now using the [Set Z Offset notebook](./0_SetZOffset.ipynb)**. You need to do this each time you turn the machine on.

### Set up our tools
We'll use a syringe tool to move liquid and a lab automation deck to hold labware.

In [ ]:
# To load the tool, specify the index of the tool on your machine
# If you're not sure what your tool index is, you can query Duet about what tools are configured like this:
m.configured_tools

In [ ]:
# Change the following tool_index variable to match your machine configuration, if necessary
syringe_index = 2

In [ ]:
# The syringe config file contains calibration data
syringe = Syringe(index = syringe_index,
                  name = 'syringe',
                  config = '10cc_syringe_liquidhandling'
                 )
m.load_tool(syringe)

In [ ]:
# Pick up the syringe!
m.pickup_tool(syringe)

# Drop the bed down so we have room to load our labware
m.move_to(z=150)

### Load our labware!
We'll use petri dishes to hold our liquid, and a 24-well plate to mix colors. Feel free to set this up without any liquid to do a 'dry run' if you'd prefer.

In particular, we will put:
- a petri dish filled with water into slot 0
- a 24 well plate into slot 1
- a petri dish filled with colored water into slot 2

In [ ]:
# We put our liquid reservoirs into slot 0 and 2 of the machine
water = m.load_labware("generic_petri_dish_100ml", 0)
color = m.load_labware("generic_petri_dish_100ml", 2)

# There's only 1 "well" in a petri dish! So we'll access them directly for convenience.
water = water["A1"]
color = color["A1"]

# We put our 24 well plate into slot 1 of the machine
samples = m.load_labware("greiner_24_wellplate_3300ul", 1)

Now, you should **physically** install the labware into the machine. Be sure to double check that you put the slots that you put each into!

### 3. Set up the sample plate
The following cells will:
- move water into all wells of the 24 well plate
- move colored water into the first column



In [ ]:
# Put decreasing amount of water into first column
fill_volume = 2  # value in mL
step_size = 0.5
for well in samples.column_data[1]:
    syringe.transfer(
        vol = fill_volume,
        source = water,
        destination = well)
    fill_volume -= step_size

In [ ]:
# Add water to all other wells
fill_volume = 1.5  # value in mL
for column in range(2, len(samples.column_data)+1):
    syringe.transfer(
        vol = fill_volume,
        source = water,
        destination = list(samples.column_data[column])
        )

In [ ]:
# Add increasing amount of colored water to the first column
fill_volume = 0.5
for well in samples.column_data[1]:
    syringe.transfer(
        vol = fill_volume,
        source = color,
        destination = well,
        mix_after=(1,2)
    )
    fill_volume += step_size

## Run Serial Dilution!
The following cell will transfer and mix samples down each column of the well plate

In [ ]:
transfer_volume = 1.5
for row in samples.row_data:
    for column in range(1, len(samples.column_data)):
        syringe.transfer(
            vol = transfer_volume,
            source = samples.wells[f"{row}{column}"],
            destination = samples.wells[f"{row}{column+1}"],
            mix_after = (1, 2)
        )